### **Imports**

In [1]:
import os
import json
from PIL import Image
import zipfile
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import zipfile

# # Specify the path to the dataset zip file
# zip_file_path = '/content/drive/MyDrive/Fashion Search Engine Project/Data/Images_data.zip'

# # Specify the target directory within Colab where you want to extract the dataset
# target_directory = '/content/drive/MyDrive/Fashion Search Engine Project/Data/Dataset_Extracted/Full_Shot_Images_Original/'

# # Extract the dataset
# with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#     zip_ref.extractall(target_directory)


### **Defined Variables**

In [3]:
project_path = '/content/drive/MyDrive/Fashion Search Engine Project'
labels_path   = project_path + '/Data/Lables.json'
src_img_path  = project_path + '/Data/Dataset_Extracted/Full_Shot_Images_Original'
dist_img_path = project_path + '/Data/Dataset_Extracted/Items_Extracted/Image_based'

In [4]:
fashion_categories = [
    'Suitcase', 'Miniskirt', 'Tie', 'Luggage & bags', 'Shoe', 'Belt', 'Outerwear', 'Dress',
    'Earrings', 'Bracelet', 'Necklace', 'Brassiere', 'Footwear',
    'Satchel', 'Bowtie', 'Top', 'Pants', 'Sunglasses', 'Swimwear', 'Clothing',
    'Glove', 'Skirt', 'High heels', 'Underpants',
    'Fedora','Sun hat', 'Sock',
    'Wallet', 'Scarf', 'Watch', 'Umbrella', 'Glasses',
    'Boot', 'Basket', 'Backpack', 'Bag',
    'Hat', 'Coat', 'Sandal', 'Shorts', 'Jeans', 'Shirt', 'Handbag', 'Jacket'
]

### **Verifing Helper Functions**

In [ ]:
def list_of_imgs(folder_path):
  return [file for file in os.listdir(folder_path)]

In [ ]:
def count_imgs_in_folder(folder_path):
  print(f"Number of images in the folder: {len(os.listdir(folder_path))}")

In [ ]:
def count_imgs_in_labels(json_file):
    with open(json_file, 'r') as f:
        annotations = json.load(f)
    lst = list(annotations.keys())
    return len(lst)

In [ ]:
#number of images in the json file
count_imgs_in_labels(labels_path) 

46089

In [ ]:
# original images cnt - fashion only
print(f"Number of images in the folder: {len(os.listdir(src_img_path))}")

Number of images in the folder: 44860


In [ ]:
def count_subfolders(folder_path):
    subfolder_count = 0

    # Iterate over each item in the folder
    for item in os.listdir(folder_path):
        item_path = os.path.join(folder_path, item)  # Get the full path of the item

        # Check if the item is a subfolder
        if os.path.isdir(item_path):
            subfolder_count += 1

    return subfolder_count

total_subfolder_count = count_subfolders(dist_img_path)
print("Total subfolder count:", total_subfolder_count)

Total subfolder count: 44860


In [ ]:
src_imgs= list_of_imgs(src_img_path)

### **Image Based Extractions**

In [ ]:
# task >> eny 23ed run l bta3a deh
def extract_items_from_annotations_image_based(json_file, image_directory, output_directory):
    with open(json_file, 'r') as f:
        annotations = json.load(f)
    cnt=0
    current_image=0

    for image_filename, items in annotations.items():
      if image_filename in src_imgs:
        # continue
        current_image +=1
        print(f'We are in image number {current_image}')
        image_path = image_directory + image_filename
        image = Image.open(image_path)

        label_counts = {}  # To keep track of label occurrences within the image
        for item in items:
            for label, coordinates in item.items():
                if label not in fashion_categories:
                    cnt+=1
                    continue  # Skip unrelated labels
                # print(f'hello fashion Item')
                if label not in label_counts:
                    label_counts[label] = 1
                    label_id = label
                else:
                    label_counts[label] += 1
                    label_id = f"{label}_{label_counts[label]}"

                x_min = int(coordinates[0] * image.width)
                y_min = int(coordinates[1] * image.height)
                x_max = int(coordinates[2] * image.width)
                y_max = int(coordinates[3] * image.height)

                item_image = image.crop((x_min, y_min, x_max, y_max))

                # Create subfolder for the image if it doesn't exist
                image_output_directory = os.path.join(output_directory, image_filename.split('.')[0])
                os.makedirs(image_output_directory, exist_ok=True)

                output_filename = f"{label_id}_{image_filename}"
                output_path = os.path.join(image_output_directory, output_filename)
                item_image.save(output_path)
    print(f'image skipped(Non Fashiom Image) : {cnt}')

In [ ]:
extract_items_from_annotations_image_based(labels_path,f'{src_img_path}/', dist_img_path)

Streaming output truncated to the last 5000 lines.
We are in image number 39862
We are in image number 39863
We are in image number 39864
We are in image number 39865
We are in image number 39866
We are in image number 39867
We are in image number 39868
We are in image number 39869
We are in image number 39870
We are in image number 39871
We are in image number 39872
We are in image number 39873
We are in image number 39874
We are in image number 39875
We are in image number 39876
We are in image number 39877
We are in image number 39878
We are in image number 39879
We are in image number 39880
We are in image number 39881
We are in image number 39882
We are in image number 39883
We are in image number 39884
We are in image number 39885
We are in image number 39886
We are in image number 39887
We are in image number 39888
We are in image number 39889
We are in image number 39890
We are in image number 39891
We are in image number 39892
We are in image number 39893
We are in image numbe

In [ ]:
count_imgs_in_folder(dist_img_path)

Number of images in the folder: 44860


### **Category Based Extraction**

In [ ]:
def extract_items_from_annotations_category_based(json_file, image_directory, output_directory):
    with open(json_file, 'r') as f:
        annotations = json.load(f)

    # Create subfolders based on fashion categories
    for category in fashion_categories:
        category_output_directory = os.path.join(output_directory, category)
        os.makedirs(category_output_directory, exist_ok=True)

    for image_filename, items in annotations.items():
        image_path = image_directory + image_filename
        image = Image.open(image_path)

        for item in items:
            for label, coordinates in item.items():
                if label not in fashion_categories:
                    continue  # Skip unrelated labels

                x_min = int(coordinates[0] * image.width)
                y_min = int(coordinates[1] * image.height)
                x_max = int(coordinates[2] * image.width)
                y_max = int(coordinates[3] * image.height)

                item_image = image.crop((x_min, y_min, x_max, y_max))

                output_filename = f"{label}_{image_filename}"
                output_path = os.path.join(output_directory, label, output_filename)
                item_image.save(output_path)

In [ ]:
dist_img_path = project_path + '/Data/Dataset_Extracted/Items_Extracted/Category_based/'

In [ ]:
extract_items_from_annotations_category_based(labels_path, src_img_path, dist_img_path)

In [ ]:
count_imgs_in_folder(dist_img_path)

### **Delete Non-Fashion Images**

In [ ]:
def remove_Non_Fashion_Images(json_file,original_images_paths):
   with open(json_file, 'r') as f:
        annotations = json.load(f)
   cnt=0
   
   for image_filename, items in annotations.items():
      trig = False
      for item in items:
            for label, coordinates in item.items():
                if label in fashion_categories:
                    trig = True
                    break
      if not trig:
        cnt+=1
        print(cnt)
        try:
          # print(os.path.join(original_images_paths, image_filename))
          os.remove(os.path.join(original_images_paths, image_filename))
        except:
          print("Image doesn't exist")
   print(f'Number of images Removed {cnt}')

In [ ]:
src_img_path  = project_path + '/Data/Dataset_Extracted/Full_Shot_Images_Original'
remove_Non_Fashion_Images(labels_path,src_img_path)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


**Checking number of images and anotations after deletion**

In [ ]:
count_imgs_in_folder(src_img_path+'/')

Number of images in the folder: 44860


### **Zipping Image Based**

In [7]:
def compress_folder(folder_path, zip_path):
    # Create a new ZIP file
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Iterate over each item in the folder
        for root, _, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)  # Get the full path of the file
                zipf.write(file_path, arcname=os.path.relpath(file_path, folder_path))



In [8]:

zip_path = project_path + "Data/Dataset_Extracted/Items_Extracted/Image_based_compressed.zip"  

# Specify the path of the ZIP file
zip_directory = os.path.dirname(zip_path)
os.makedirs(zip_directory, exist_ok=True)

compress_folder(dist_img_path, zip_path)